In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
src_path = Path('/content/drive/MyDrive/P9')

In [ ]:
clicks = pd.read_csv(src_path / 'clicks.csv')

small_clicks = clicks[clicks.user_id.isin(range(0,10000))]

print(small_clicks.shape)
small_clicks.head()

(225648, 13)


,user_id,session_id,session_start,session_size,article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type,filename
0,0,1506825423271737,2017-10-01 02:37:03,2,157541,2017-10-01 03:00:28.020,4,3,20,1,20,2,0
1,0,1506825423271737,2017-10-01 02:37:03,2,68866,2017-10-01 03:00:58.020,4,3,20,1,20,2,0
2,1,1506825426267738,2017-10-01 02:37:06,2,235840,2017-10-01 03:03:37.951,4,1,17,1,16,2,0
3,1,1506825426267738,2017-10-01 02:37:06,2,96663,2017-10-01 03:04:07.951,4,1,17,1,16,2,0
4,2,1506825435299739,2017-10-01 02:37:15,2,119592,2017-10-01 03:04:50.575,4,1,17,1,24,2,0


In [ ]:
small_clicks.article_id.nunique()

12423

In [ ]:
small_clicks.to_csv(src_path / 'small_clicks.csv', index=False)

In [ ]:
embeddings = pd.read_pickle(src_path / 'articles_embeddings.pickle')
df_embeddings = pd.DataFrame(embeddings)

# Build the article IDs list
list_articleID = sorted(list(small_clicks.article_id.unique()))

test_embeddings = df_embeddings[df_embeddings.index.isin(list_articleID)]

small_embeddings = test_embeddings.to_numpy()
small_embeddings.shape

(12423, 250)

In [ ]:
# Save as Pickle
pickle.dump(small_embeddings, open(src_path / 'small_embeddings.pickle', 'wb'))

small_embeddings = pd.read_pickle(src_path / 'small_embeddings.pickle')

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_ContentBased_Reco(userID, small_clicks, small_embeddings, n_reco=5):
    '''Return 5 recommended articles ID to user'''

    # Get the list of articles viewed by the user
    var = small_clicks.loc[small_clicks.user_id == userID]['article_id'].to_list()

    # Get the list of unique article_ID in small_clicks
    list_articleID = sorted(list(small_clicks.article_id.unique()))

    # Retrieve the corresponding index of the articles viewed by userID in var
    idx_var = []
    for i in range(0, len(var)):
        for idx, item in enumerate(list(list_articleID)):
            if item == var[i]:
                idx_var.append(idx)

    # Select the last element of the list
    value = idx_var[-1]
    # print(value)

    # Compute the cosine similarity
    emb = small_embeddings
    distances = cosine_similarity([emb[value]], emb)[0]

    # Save the result in Pandas DataFrame
    df_reco = pd.DataFrame(list(zip(list_articleID, distances)),
                           columns=(["reco_article_id", "similarity"]))
    
    # Sort the DF by similarity
    df_reco.sort_values(by='similarity', ascending=False, inplace=True)

    # Exclude already viewed articles
    top_reco = df_reco[~df_reco.reco_article_id.isin(var)]

    # Give the list of recommended articles
    result = list(top_reco["reco_article_id"].iloc[:(n_reco)].values)

    return result

In [ ]:
#### Test the function ####
# Choose a userID
userID = 20

# Save the function in a variable
reco5 = get_ContentBased_Reco(userID, small_clicks, small_embeddings, n_reco=5)

# Display the result
reco5

[70397, 157507, 157865, 157496, 157802]